**TFIDF VECTORIZAR,CLASSIFIER: MLKNN**

In [1]:
#needed installs
!pip install scikit-learn==0.24.1
!pip install scikit-multilearn

#needed imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.adapt import MLkNN
from sklearn.metrics import hamming_loss, accuracy_score

#mount drive
from google.colab import drive
drive.mount('/content/drive')
print('mount success')

#read from file
aspects_df = pd.read_csv('/content/drive/Shareddrives/EMOThreat/train_set_taskA.csv')
X = aspects_df["Sentences"]
y = np.asarray(aspects_df[aspects_df.columns[:7]])
  
# initializing TfidfVectorizer 
vetorizar = TfidfVectorizer(max_features=7000, max_df=0.85)
# fitting the tf-idf on the given data
vetorizar.fit(X)
  
# splitting the data to training and testing data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
  
# transforming the data
X_train_tfidf = vetorizar.transform(X_train)
X_test_tfidf = vetorizar.transform(X_test)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 22.3 MB 1.1 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 5.3 MB/s 
Mounted at /content/drive
mount success


In [ ]:

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

#using Multi-label kNN classifier  
import time 
from sklearn.model_selection import GridSearchCV
start=time.time()

mlknn_classifier = MLkNN(k=20)
#print(X_train_tfidf.shape,"\n\n")
mlknn_classifier.fit(X_train_tfidf, y_train)
predicted = mlknn_classifier.predict(X_test_tfidf)

print("Hamming Lossing:",hamming_loss(y_test, predicted))
print("Accuracy Score:",accuracy_score(y_test, predicted))
print('Training Time Taken: ',round(time.time()-start,0),'seconds')

Hamming Lossing: 0.10476190476190476
Accuracy Score: 0.5717948717948718
Training Time Taken:  15.0 seconds


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix,classification_report
from sklearn.metrics import precision_score, recall_score, f1_score

print(classification_report(y_test, predicted))
print(multilabel_confusion_matrix(y_test, predicted))

from sklearn.metrics import accuracy_score
print("ACC: ",accuracy_score(predicted,y_test))

              precision    recall  f1-score   support

           0       0.71      0.32      0.44        93
           1       0.63      0.25      0.36        88
           2       0.94      0.30      0.46        56
           3       0.70      0.60      0.65       234
           4       0.60      0.42      0.49       149
           5       0.54      0.38      0.44        95
           6       1.00      0.98      0.99       312

   micro avg       0.79      0.60      0.68      1027
   macro avg       0.73      0.46      0.55      1027
weighted avg       0.77      0.60      0.66      1027
 samples avg       0.69      0.67      0.66      1027

[[[675  12]
  [ 63  30]]

 [[679  13]
  [ 66  22]]

 [[723   1]
  [ 39  17]]

 [[486  60]
  [ 94 140]]

 [[589  42]
  [ 87  62]]

 [[654  31]
  [ 59  36]]

 [[468   0]
  [  5 307]]]
ACC:  0.5717948717948718


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pandas as pd
aspects_df = pd.read_csv('/content/drive/Shareddrives/EMOThreat/test_set_taskA.csv')
X = aspects_df["Sentences"]
xx=X
vetorizar = TfidfVectorizer(max_features=7000, max_df=0.85)
# fitting the tf-idf on the given data
vetorizar.fit(X)
X = vetorizar.transform(X)
#knn_predictions = knn_classifier.predict(X)
#dnn_predictions=dnn_classifier.predict(X)
#rf_predictions=rf_classifier.predict(X)
mlknn=mlknn_classifier.predict(X)
csv = pd.DataFrame(mlknn.toarray(),columns=["anger","disgust","fear","sadness","surprise","happiness","neutral"])
csv.insert(7,"Sentences",aspects_df["Sentences"],True)
csv.to_csv("pred_tfidf_mlknn.csv", index=False)